In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

In [2]:

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path


import pandas as pd
import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.solver_model import MultiAgentSolverModel
from multi_agent.colaboration_types import ColaborationTypes
from solver_step import SolverStep
from multi_agent.solution_pool import solution_pool_simple
%matplotlib widget

In [3]:
truck_volume_capacity = 30
truck_weight_capacity = 20000
truck_capacity = (truck_volume_capacity, truck_weight_capacity)
dists, demand, points = generator(40)

In [4]:
customer_df = pd.read_excel('data/2_detail_table_customers.xls')
deposit_df = pd.read_excel('data/4_detail_table_depots.xls')
routes = customer_df['ROUTE_ID'].unique()
dists, demand, points = format_input(routes[0],customer_df,deposit_df)

In [5]:
routes

array([2946091, 2922001, 2939484, 2958047, 2970877, 2990001, 3005971,
       2604001, 3016355, 3027038, 3044702])

In [6]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [7]:
solver_step_SA = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [8]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    taboo_list_size
    )

In [9]:
init_state = nearest_neighbour_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),100)
step_function_list = [solver_step_SA, solver_step_TS]

In [10]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(lambda : init_step, step_function_list, 2946091,20000,30,solution_pool,GA_stepSize=20,colaboration_type=ColaborationTypes.ENEMIES)

/opt/miniconda3/lib/python3.12/site-packages/mesa/time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


In [11]:
for _ in range(100):
    sol_model.step()
    for sol in sol_model.solution_pool.pool:
        print(sol.get_best_sol())

population generated
[0, 1, 0, 68, 0, 10, 0, 93, 85, 0, 19, 0, 24, 98, 0, 94, 0, 7, 0, 83, 0, 45, 0, 65, 18, 0, 56, 0, 60, 0, 69, 0, 9, 0, 48, 0, 46, 0, 67, 0, 25, 0, 16, 0, 71, 0, 80, 0, 101, 0, 62, 0, 20, 0, 66, 0, 73, 0, 84, 0, 58, 0, 37, 0, 40, 0, 2, 81, 0, 89, 0, 61, 0, 57, 0, 42, 0, 70, 0, 41, 0, 99, 0, 5, 0, 82, 0, 38, 0, 49, 0, 72, 0, 103, 0, 44, 0, 12, 54, 55, 0, 17, 0, 36, 0, 63, 0, 28, 0, 91, 0, 59, 0, 21, 0, 90, 0, 79, 0, 86, 0, 104, 0, 97, 102, 0, 87, 3, 96, 0, 23, 52, 0, 47, 0, 106, 0, 50, 0, 15, 0, 88, 0, 35, 0, 39, 0, 78, 0, 6, 0, 77, 0, 8, 0, 74, 0, 29, 0, 33, 0, 64, 0, 4, 31, 0, 95, 0, 105, 107, 0, 13, 0, 11, 22, 0, 43, 0, 100, 0, 30, 0, 32, 0, 34, 51, 0, 76, 0, 75, 26, 53, 92, 14, 0, 27, 0]
[0, 1, 0, 45, 0, 48, 82, 0, 85, 0, 38, 69, 46, 0, 19, 0, 24, 98, 71, 0, 70, 0, 56, 57, 0, 0, 9, 10, 61, 0, 25, 0, 0, 17, 0, 93, 0, 16, 0, 0, 42, 0, 66, 0, 20, 94, 0, 0, 62, 0, 67, 0, 101, 0, 80, 0, 68, 0, 0, 54, 0, 0, 44, 0, 12, 41, 84, 0, 0, 60, 89, 0, 40, 0, 2, 81, 63, 0, 36, 0,

KeyboardInterrupt: 

In [9]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states['TheGlobalBest'][0])

[0, 6, 33, 36, 30, 37, 12, 38, 32, 18, 22, 0, 28, 15, 23, 27, 11, 35, 31, 14, 19, 1, 0, 29, 17, 3, 20, 5, 25, 39, 21, 16, 10, 0, 4, 9, 34, 8, 24, 26, 7, 2, 13, 0]
